In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
device = "cpu"

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(device)

cuda


In [3]:
input_size = 5  # 5
hidden_size = 50
num_layers = 2
output_size = 1
learning_rate = 0.001
num_epochs = 100
batch_size = 64 # 배치 크기 설정

filename = "KRW-XRP-1m-full"

model_filename = f"{filename}-lr{learning_rate}_bs{batch_size}-epochs{num_epochs}-hs{hidden_size}_nl{num_layers}.pth"


print("hidden_size:", hidden_size)
print("num_layers:", num_layers)
print("num_epochs:", num_epochs)
print("batch_size:", batch_size)

hidden_size: 50
num_layers: 2
num_epochs: 100
batch_size: 64


In [4]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, device):
        super(LSTMModel, self).__init__()
        
        self.device = device
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM 레이어 정의
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # 출력 레이어 정의
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # 초기 hidden state와 cell state 설정
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=self.device).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=self.device).requires_grad_()

        # LSTM 실행
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # 마지막 시점의 출력값 사용
        out = self.fc(out[:, -1, :])
        return out

In [5]:
# 모델 로드 전에 state_dict 수정
state_dict = torch.load(f"../model/{model_filename}")
# "module." 접두어를 제거하는 작업

# 모델 생성
model = LSTMModel(input_size, hidden_size, num_layers, output_size, device).to(device)

# 수정된 state_dict를 모델에 적용
model.load_state_dict(state_dict)

# 모델을 평가 모드로 설정
model.eval()

# model = LSTMModel(input_size, hidden_size, num_layers, output_size,device).to(device)

# # Load the state_dict from the .pth file
# model.load_state_dict(torch.load(f"../model/{model_filename}", weights_only=True))

# # Set the model to evaluation mode
# model.eval()

# # Now the model is ready to be used for inference

/tmp/ipykernel_2765578/3620911080.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f"../model/{model_filename}")


LSTMModel(
  (lstm): LSTM(5, 50, num_layers=2, batch_first=True)
  (fc): Linear(in_features=50, out_features=1, bias=True)
)

In [6]:
X = np.load(f"../preprocessed/{filename}-test-X.npy")
y = np.load(f"../preprocessed/{filename}-test-y.npy")

X_test = torch.from_numpy(X).type(torch.FloatTensor).to(device)
y_test = torch.from_numpy(y).type(torch.FloatTensor).to(device)

In [ ]:
print(len(X_test))
if torch.cuda.is_available():
    torch.cuda.empty_cache()

with torch.no_grad():
    train_predict = model(X_test)

199940


OutOfMemoryError: CUDA out of memory. Tried to allocate 47.75 GiB. GPU 0 has a total capacity of 47.43 GiB of which 41.40 GiB is free. Process 4095890 has 0 bytes memory in use. Including non-PyTorch memory, this process has 0 bytes memory in use. Of the allocated memory 3.13 GiB is allocated by PyTorch, and 15.60 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
df_origin = pd.read_csv(f"../data/{filename}-test.csv")
# 필요한 열 선택
df = df_origin[['candle_date_time_kst', 'opening_price', 'high_price', 'low_price', 'trade_price', 'candle_acc_trade_volume']]

# datetime 형식으로 변환
df['candle_date_time_kst'] = pd.to_datetime(df['candle_date_time_kst'])
df.set_index('candle_date_time_kst', inplace=True)

# 결측치 제거
df.dropna(inplace=True)


import joblib
scaler = joblib.load('../util/KRW-XRP-1m-full.pkl')
scaled_df = scaler.transform(df)

# DataFrame으로 변환
scaled_df = pd.DataFrame(scaled_df, index=df.index, columns=df.columns)

In [12]:
# 예측값과 실제값을 스케일러의 inverse_transform을 사용하여 원래 값으로 복원

# 예측값
predicted = train_predict.detach().cpu().numpy()
# 실제값
actual = y_test.detach().cpu().numpy()

# trade_price만 복원하기 위해 다른 컬럼은 0으로 채움
padding = np.zeros((predicted.shape[0], scaled_df.shape[1]-1))
predicted_full = np.concatenate((padding, predicted), axis=1)
actual_full = np.concatenate((padding, actual.reshape(-1,1)), axis=1)

# inverse_transform
predicted_original = scaler.inverse_transform(predicted_full)[:, -1]
actual_original = scaler.inverse_transform(actual_full)[:, -1]

In [ ]:
print(actual)
print(actual_full)
print(actual_original)

print("@@")

print(predicted)
print(predicted_original)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df.index[-len(predicted_original):], actual_original, label='Actual Price')
plt.plot(df.index[-len(predicted_original):], predicted_original, label='Predicted Price')
plt.legend()
plt.show()

In [29]:
# 가격 상승 예측 시 매수, 하락 예측 시 매도
signals = []
for i in range(len(predicted_original)-1):
    if predicted_original[i+1] > predicted_original[i]:
        signals.append('Buy')
    else:
        signals.append('Sell')

In [ ]:
import pandas as pd



signal_df = pd.DataFrame({
    'Date': df.index[-len(signals):],
    'Actual Price': actual_original[:-1],
    'Predicted Price': predicted_original[:-1],
    'Signal': signals
})

pd.set_option('display.max_rows', None)  # 모든 행을 출력하도록 설정
print(signal_df)

print(signal_df.head())

In [ ]:
import pandas as pd

# 초기 자본 및 변수 설정
initial_capital = 1000000  # 100만원
cash = initial_capital  # 초기 현금
coins = 0  # 보유 코인 수
asset_history = []  # 자산 변동 이력

# 거래 로직 실행
for index, row in signal_df.iterrows():
    current_price = row['Predicted Price']  # 현재 가격 사용
    if row['Signal'] == 'Buy' and cash > 0:  # 매수 조건
        coins = cash / current_price  # 모든 현금으로 코인 매수
        cash = 0  # 현금 소모
    elif row['Signal'] == 'Sell' and coins > 0:  # 매도 조건
        cash = coins * current_price  # 모든 코인 매도
        coins = 0  # 코인 소모

    # 현재 자산 계산 (현금 + 코인 가치)
    total_assets = cash + coins * current_price
    asset_history.append(total_assets)  # 자산 이력 기록

# 자산 이력을 DataFrame으로 변환
asset_df = pd.DataFrame(asset_history, columns=['Total Assets'])
asset_df['Date'] = signal_df['Date']  # 날짜 정보 추가

# 결과 출력
print(asset_df)
print(f"First Total Assets: {asset_df['Total Assets'].iloc[0]:.0f}")
print(f"Last Total Assets: {asset_df['Total Assets'].iloc[-1]:.0f}")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(asset_df['Date'], asset_df['Total Assets'], label='Total Assets')
plt.xlabel('Date')
plt.ylabel('Assets in KRW')
plt.title('Asset Variation Over Time')
plt.legend()
plt.grid(True)
plt.show()

# 이평선

In [ ]:
# 이동 평균 계산
short_window = 40
long_window = 100

signal_df['short_mavg'] = signal_df['Predicted Price'].rolling(window=short_window, min_periods=1, center=False).mean()
signal_df['long_mavg'] = signal_df['Predicted Price'].rolling(window=long_window, min_periods=1, center=False).mean()

# 매수/매도 신호 생성
signal_df['signal'] = 0
signal_df['signal'][short_window:] = np.where(signal_df['short_mavg'][short_window:] > signal_df['long_mavg'][short_window:], 1, 0)
signal_df['positions'] = signal_df['signal'].diff()

# 시각화
plt.figure(figsize=(16, 7))
plt.plot(signal_df['Predicted Price'], label='Predicted Price')
plt.plot(signal_df['short_mavg'], label='40-Day Moving Average')
plt.plot(signal_df['long_mavg'], label='100-Day Moving Average')

# 매수 시그널 표시
plt.plot(signal_df[signal_df['positions'] == 1].index, 
         signal_df['short_mavg'][signal_df['positions'] == 1], 
         '^', markersize=10, color='g', lw=0, label='Buy Signal')

# 매도 시그널 표시
plt.plot(signal_df[signal_df['positions'] == -1].index, 
         signal_df['short_mavg'][signal_df['positions'] == -1], 
         'v', markersize=10, color='r', lw=0, label='Sell Signal')

plt.title('Predicted Price and Moving Averages')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 초기 설정
initial_capital = 1000000  # 초기 자본: 100만원
cash = initial_capital
coins = 0
asset_history = []

# 이동 평균 계산
signal_df['short_mavg'] = signal_df['Predicted Price'].rolling(window=5, min_periods=1).mean()
signal_df['long_mavg'] = signal_df['Predicted Price'].rolling(window=20, min_periods=1).mean()

# 매수/매도 신호 생성
signal_df['signal'] = 0
signal_df['signal'][40:] = np.where(signal_df['short_mavg'][40:] > signal_df['long_mavg'][40:], 1, 0)
signal_df['positions'] = signal_df['signal'].diff()

# 거래 로직
for index, row in signal_df.iterrows():
    # 매수 신호
    if row['positions'] == 1:
        if cash > 0:  # 현금이 있을 때만 매수
            coins = cash / row['Predicted Price']
            cash = 0
    # 매도 신호
    elif row['positions'] == -1:
        if coins > 0:  # 코인이 있을 때만 매도
            cash = coins * row['Predicted Price']
            coins = 0
    
    # 자산 업데이트
    total_assets = cash + coins * row['Predicted Price']
    asset_history.append(total_assets)

# 자산 이력 데이터 프레임 생성
assets_df = pd.DataFrame(asset_history, index=signal_df.index, columns=['Total Assets'])

# 결과 시각화
plt.figure(figsize=(12, 6))
plt.plot(assets_df['Total Assets'], label='Total Assets')
plt.title('Asset Variation Over Time Using Improved Trading Strategy')
plt.xlabel('Date')
plt.ylabel('Total Assets in KRW')
plt.legend()
plt.grid(True)
plt.show()

# 자산 최종 결과 출력
final_assets = assets_df.iloc[-1]
return_percentage = ((final_assets / initial_capital) - 1) * 100
print(f"Final assets: {final_assets['Total Assets']:.2f} KRW")
print(f"Return on investment: {return_percentage['Total Assets']:.2f}%")

In [ ]:
import numpy as np
import pandas as pd

# 가정: signal_df에는 'Predicted Price' 컬럼과 날짜 인덱스가 있다고 가정
# 또한 학습을 위한 실제 모의 거래는 'Predicted Price'로만 진행하고, 
# short_window와 long_window를 변경해가며 성능 평가를 함.

initial_capital = 1000000  # 초기 자본: 100만원

best_final_assets = -1
best_params = (None, None)  # (short_window, long_window)

# short_window와 long_window의 범위 정의(예: 5~50, 20~100)
for short_window in range(5, 51, 5):   # 예: 5,10,15,...,50
    for long_window in range(short_window+5, 101, 5):  # 장기 이평은 단기보다 충분히 길게, 예: 단기+5부터 100까지 5단위
        # DataFrame 복사 (원본 손상 방지)
        temp_df = signal_df.copy()
        
        # 이동 평균 계산
        temp_df['short_mavg'] = temp_df['Predicted Price'].rolling(window=short_window, min_periods=1).mean()
        temp_df['long_mavg'] = temp_df['Predicted Price'].rolling(window=long_window, min_periods=1).mean()

        # 매수/매도 신호 생성
        temp_df['signal'] = 0
        # 충분한 데이터 확보를 위해 long_window 이후부터 신호 생성
        temp_df.loc[temp_df.index[long_window:], 'signal'] = np.where(
            temp_df['short_mavg'][long_window:] > temp_df['long_mavg'][long_window:], 1, 0
        )
        temp_df['positions'] = temp_df['signal'].diff()

        # 거래 로직
        cash = initial_capital
        coins = 0
        asset_history = []

        for index, row in temp_df.iterrows():
            current_price = row['Predicted Price']
            
            # 매수 신호
            if row['positions'] == 1:
                if cash > 0:  # 현금이 있을 때 매수
                    coins = cash / current_price
                    cash = 0
            # 매도 신호
            elif row['positions'] == -1:
                if coins > 0:
                    cash = coins * current_price
                    coins = 0
            
            total_assets = cash + coins * current_price
            asset_history.append(total_assets)
        
        final_assets = asset_history[-1] if asset_history else initial_capital
        
        # 최적 조합 갱신
        if final_assets > best_final_assets:
            best_final_assets = final_assets
            best_params = (short_window, long_window)

print(f"Best short_window: {best_params[0]}, Best long_window: {best_params[1]}")
print(f"Final assets with best params: {best_final_assets} KRW")
print(f"Return: {((best_final_assets / initial_capital) - 1) * 100:.2f}%")